In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.optimize import minimize
import joblib
import pickle
# Load dataset
def load_data():
    data = pd.read_csv('/content/drive/MyDrive/datasets/gym_members_exercise_tracking.csv')
    return data

# Encode categorical columns
def encode_data(data):
    label_encoders = {}
    categorical_columns = ['Gender', 'Workout_Type', 'Fitness_Goal']
    for column in categorical_columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le
    return data, label_encoders

# Model for Level 1: Fitness Goal Prediction (RandomForestClassifier)
def train_fitness_goal_classifier(data):
    level1_features = ['Age', 'Gender', 'BMI', 'Workout_Frequency (days/week)', 'Experience_Level', 'Workout_Type', 'Fat_Percentage']
    level1_target = 'Fitness_Goal'

    X1 = data[level1_features]
    y1 = data[level1_target]

    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42, stratify=y1)

    clf = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=5, min_samples_split=10, min_samples_leaf=5)
    clf.fit(X1_train, y1_train)

    # Save the model to a file
    joblib.dump(clf, 'fitness_goal_classifier.pkl')
    return clf


# # بعد تدريب النموذج
# # حفظ نموذج التصنيف (Fitness Goal Classifier)
#     with open('fitness_goal_classifier.pkl', 'wb') as f:
#          pickle.dump(clf, f)


# Model for Level 2: Calories Burned Prediction (RandomForestRegressor)
def train_calories_burned_regressor(data, label_encoders):
    level2_features = ['Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Max_BPM', 'Avg_BPM', 'Resting_BPM', 'Session_Duration (hours)', 'Workout_Type', 'Fat_Percentage', 'BMI', 'Fitness_Goal']
    level2_target = 'Calories_Burned'

    # Add Fitness Goal to features for level 2
    data['Fitness_Goal'] = data['Fitness_Goal'].apply(lambda x: label_encoders['Fitness_Goal'].inverse_transform([x])[0])

    X2 = data[level2_features]
    y2 = data[level2_target]

    scaler = StandardScaler()
    X2 = scaler.fit_transform(X2)

    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

    regressor = RandomForestRegressor(random_state=42, n_estimators=300, max_depth=20, min_samples_split=5, min_samples_leaf=2)
    regressor.fit(X2_train, y2_train)

    # Save the regressor and scaler to files
    joblib.dump(regressor, 'calories_burned_regressor.pkl')

    joblib.dump(scaler, 'scaler.pkl')

    return regressor, scaler
# # حفظ نموذج الانحدار (Calories Burned Regressor)
#     with open('calories_burned_regressor.pkl', 'wb') as f:
#          pickle.dump(regressor, f)

# # حفظ StandardScaler (إذا كان قد تم استخدامه)
#     with open('scaler.pkl', 'wb') as f:
#          pickle.dump(scaler, f)
# Function for Level 3 Optimization (Workout Plan)
def objective(workout_plan, alpha=0.6, beta=0.3, gamma=0.1, user_experience=1, predicted_calories=1, fitness_goal=None):
    session_duration, target_bpm, fatigue_risk = workout_plan
    adjusted_alpha = alpha * user_experience
    adjusted_beta = beta * (target_bpm / 200)
    adjusted_gamma = gamma * (1 + fatigue_risk)

    goal_alignment = 0
    if fitness_goal == 'Weight_Loss':
        goal_alignment = 1
    elif fitness_goal == 'Muscle_Gain':
        goal_alignment = 0.8
    else:
        goal_alignment = 0.9

    penalty_duration = 0 if session_duration <= 1 + user_experience * 0.2 else session_duration * 0.5

    return -(adjusted_alpha * predicted_calories * goal_alignment - adjusted_beta - adjusted_gamma + penalty_duration)

def constraint_duration(workout_plan, user_experience):
    session_duration = workout_plan[0]
    max_duration = 0.5 + user_experience * 0.3
    return max_duration - session_duration

def constraint_bpm(workout_plan, max_bpm, age):
    target_bpm = workout_plan[1]
    safe_bpm = max_bpm - (0.5 * (age - 30))
    return safe_bpm - target_bpm

def calculate_optimal_plan(user_experience=1, max_bpm=200, predicted_calories=1, age=30, fitness_goal=None):
    initial_plan = [np.random.uniform(0.5, 2.0), np.random.uniform(100, 180), np.random.uniform(0.0, 1.0)]
    bounds = [(0.5, 2.0), (100, 200), (0.0, 1.0)]

    result = minimize(objective, initial_plan, bounds=bounds,
                      constraints=[
                          {"type": "ineq", "fun": lambda workout_plan: constraint_duration(workout_plan, user_experience)},
                          {"type": "ineq", "fun": lambda workout_plan: constraint_bpm(workout_plan, max_bpm, age)}
                      ], args=(0.6, 0.3, 0.1, user_experience, predicted_calories, fitness_goal))
    return result.x

def calculate_water_intake(weight, session_duration):
    base_water = 0.03 * weight
    extra_water = session_duration * 0.5
    return base_water + extra_water




